<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>False Political Claim Detection</h1>
    <h3>Carga y limpieza de los datos - LLM</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar librerias](#1.-Importar-librerias)  
2. [Funciones auxiliares](#2.-Funciones-auxiliares)  
3. [Configuracion de los diccionarios](#3.-configuracion-de-los-diccionarios)  
4. [Creacion de One-Hot](#4.-Creacion-de-One-Hot)  
5. [Exportar CSV](#5.-Exportar-CSV)  
6. [Referencias](#6-referencias)

# 1. Importar Librerias

In [57]:
import pandas as pd
import numpy as np

from datetime import datetime
import json


import matplotlib.pyplot as plt
import seaborn as sns


import spacy

import re
from multiprocessing import process
from transformers import pipeline


from collections import Counter


# pip install gensim
import gensim
from gensim.corpora import Dictionary

# pip install pyLDAvis==3.4.1
# import pyLDAvis

# 2. Carga del Dataset

In [58]:
# Train
url = "formated/train_exportado.csv" 
df_train_exportado = pd.read_csv(url)


# Test
url2 = "formated/test_exportado.csv"
df_test_exportado = pd.read_csv(url2)

# 3. Configuracion de los diccionarios

In [59]:
statement = 'statement-lemmatize'

In [60]:
# Lista de valores mínimos de frecuencia. Una palabra debe aparecer al menos este número de veces en los documentos para ser incluida en el diccionario.
# min_freq_values = [5, 10, 20, 50]
min_freq_values = [10, 20]

# Lista de proporciones máximas. Una palabra no debe aparecer en más del porcentaje especificado de documentos para ser incluida en el diccionario.
# max_prop_values = [0.5, 0.65, 0.8]
max_prop_values = [0.8]


In [61]:
# Separar en dos listas según la etiqueta
statements_true = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 1]
statements_false =[str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 0]

print(len(statements_true))
print(len(statements_false))

5795
3155


In [62]:
# Cuenta en cuántos documentos aparece cada palabra
def document_frequency(tokenized_docs):
    df_counter = Counter()
    for doc in tokenized_docs:
        unique_words = set(doc)
        df_counter.update(unique_words)
    return df_counter

# Función para filtrar palabras por frecuencia mínima y máxima proporción
def filtra(dic_frecuencia, doc_frecuencia, total_docs, min_freq, max_prop):
    return {
        word: dic_frecuencia[word]  # frecuencia total (recuento)
        for word in dic_frecuencia
        if doc_frecuencia[word] >= min_freq and (doc_frecuencia[word] / total_docs) <= max_prop
    }

# Tokenización según clase
statements_true = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 1]
statements_false = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 0]

# Estadísticas base
n_docs_true = len(statements_true)
n_docs_false = len(statements_false)
adjust_ratio = n_docs_true / n_docs_false

# Contadores
counter_true = Counter(word for doc in statements_true for word in doc)
counter_false = Counter(word for doc in statements_false for word in doc)
df_true = document_frequency(statements_true)
df_false = document_frequency(statements_false)

# Procesa combinaciones
for min_freq_true in min_freq_values:
    min_freq_false = max(1, round(min_freq_true / adjust_ratio))  # Ajuste proporcional

    for max_prop in max_prop_values:
        # Filtrado
        filtered_true = filtra(counter_true, df_true, n_docs_true, min_freq_true, max_prop)
        filtered_false = filtra(counter_false, df_false, n_docs_false, min_freq_false, max_prop)

        # Palabras en común
        common_words = set(filtered_true.keys()) & set(filtered_false.keys())

        # Exclusivos (frecuencia absoluta)
        exclusive_true = {
            word: freq for word, freq in filtered_true.items() if word not in common_words
        }
        exclusive_false = {
            word: freq for word, freq in filtered_false.items() if word not in common_words
        }

        # Opcional: ordenar por frecuencia descendente
        filtered_true = dict(sorted(filtered_true.items(), key=lambda x: x[1], reverse=True))
        filtered_false = dict(sorted(filtered_false.items(), key=lambda x: x[1], reverse=True))
        exclusive_true = dict(sorted(exclusive_true.items(), key=lambda x: x[1], reverse=True))
        exclusive_false = dict(sorted(exclusive_false.items(), key=lambda x: x[1], reverse=True))

        # Guarda resultados
        base = f"nb{min_freq_true}_na{int(max_prop * 100)}"
        with open(f"json/dictionaries/true_{base}.json", "w") as f:
            json.dump(filtered_true, f, indent=2)
        with open(f"json/dictionaries/false_{base}.json", "w") as f:
            json.dump(filtered_false, f, indent=2)
        with open(f"json/dictionaries/true_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_true, f, indent=2)
        with open(f"json/dictionaries/false_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_false, f, indent=2)

        print(f"✔ Guardado con min_freq_true={min_freq_true}, min_freq_false={min_freq_false}, max_prop={max_prop}")


✔ Guardado con min_freq_true=10, min_freq_false=5, max_prop=0.8
✔ Guardado con min_freq_true=20, min_freq_false=11, max_prop=0.8


# 4. Creacion de One-Hot

In [63]:
# ...existing code...

def one_hot_encode_statements(statements_raw, selected_words):
    """
    Genera una codificación One-Hot para los statements, basada en un conjunto de palabras seleccionadas.

    Args:
        statements_raw (list of str): Lista de frases originales.
        selected_words (set): Conjunto de palabras seleccionadas.

    Returns:
        pd.DataFrame: DataFrame One-Hot codificado.
    """
    # Agregar prefijo 'word_' a cada palabra seleccionada
    prefixed_words = {f"word_{word}" for word in selected_words}

    # Inicializa lista para cada fila codificada
    encoded_rows = []

    for statement in statements_raw:
        tokens = set(str(statement).split())  # Convierte en set para rapidez
        encoded_row = {f"word_{word}": int(word in tokens) for word in selected_words}
        encoded_rows.append(encoded_row)

    # Crea DataFrame
    df_one_hot = pd.DataFrame(encoded_rows)
    df_one_hot.fillna(0, inplace=True)  # Por si alguna palabra no se encuentra

    return df_one_hot

In [64]:
dictionary_true = 'true_exclusive_nb20_na80'
dictionary_false = 'false_exclusive_nb20_na80'

# Carga los diccionarios true y false
with open(f"json/dictionaries/{dictionary_true}.json", 'r') as f:
    dict_true = json.load(f)

with open(f"json/dictionaries/{dictionary_false}.json", 'r') as f:
    dict_false = json.load(f)

# Combina ambos diccionarios
complete_dictionary = set({**dict_true, **dict_false}.keys())

In [65]:
df_train_one_hot = one_hot_encode_statements(df_train_exportado[statement], complete_dictionary)
df_test_one_hot = one_hot_encode_statements(df_test_exportado[statement], complete_dictionary)

In [66]:
print(df_train_one_hot.columns)

Index(['word_previous', 'word_commit', 'word_worth', 'word_face',
       'word_things', 'word_johnson', 'word_statewide', 'word_policies',
       'word_presidents', 'word_amount',
       ...
       'word_rise', 'word_some', 'word_criminal', 'word_officers', 'word_list',
       'word_union', 'word_cities', 'word_improve', 'word_fewer',
       'word_providence'],
      dtype='object', length=180)


# 5. Exportar CSV

In [67]:
# Train
url = "formated/train_exportado.csv" 
df_train = pd.read_csv(url)


# Test
url2 = "formated/test_exportado.csv"
df_test = pd.read_csv(url2)

In [68]:
# Combinar los datasets originales con sus correspondientes codificaciones one-hot
df_train_combined = pd.concat([df_train, df_train_one_hot], axis=1)
df_test_combined = pd.concat([df_test, df_test_one_hot], axis=1)


In [69]:
# Comparar las columnas de dfc y dfa
columns_train = set(df_train_combined.columns)
columns_test = set(df_test_combined.columns)

# Encontrar las diferencias
differences = columns_train.symmetric_difference(columns_test)

# Verificar si la única diferencia es 'id' y 'label'
if differences == {'id', 'label'}:
    print("La única diferencia entre las columnas es 'id' y 'label'.")
else:
    print("Existen otras diferencias en las columnas:", differences)
    print("Columnas en train pero no en test:", columns_train - columns_test)
    print("Columnas en test pero no en train:", columns_test - columns_train)

Existen otras diferencias en las columnas: {'label'}
Columnas en train pero no en test: {'label'}
Columnas en test pero no en train: set()


In [70]:
# Exportar los datasets combinados a archivos CSV
df_train_combined.to_csv('./formated/train_exportado_words.csv', index=False)
df_test_combined.to_csv('./formated/test_exportado_words.csv', index=False)

print("Datasets combinados y exportados correctamente.")

Datasets combinados y exportados correctamente.


# 5. Referencias

* [pandas documentation — pandas 2.2.3 documentation. (s. f.).](https://pandas.pydata.org/docs/)  